We use NTRU-HPS, so $q$ is a power of $2.$ We shall generate many polynomials $u(x)$. Assuming that we have an oracle that provides the length of $u_i$ and if ${\rm{len}}(u_i)={\rm{bits}}(q)-1$ provides also the second MSB, we shall compute (experimentally) the entropy of the previous random variable (if we consider that the output of the oracle is modelled by some random variable). We assume that the bits of the coefficients $\{u_i\}$ are uniformly distributed.

First, we need to generate the parameters of NTRU-HPS

In [1]:
reset()

In [2]:
def bits(N):
    if N==0:
        return 0
    else:
        return floor(log(N,2))+1

def pad_binary(bin_str):
    return '0' * (11 - len(bin_str)) + bin_str    

def count_equal(L1,L2):
    return sum(x == y for x, y in zip(L1, L2))

def correction_of_msg(N,m):
    '''
    N: the parameter of NTRU
    m: the message polynomial
    Output: a list of length N with the coefficients of m(x)
    '''
    m_list=m.list()
    if len(m_list)<N:
        diff = N - len(m_list)
        m_list.append(diff*[0])
    return flatten(m_list)

def random32(): return randrange(-2^31,2^31)        # a random integer in -2^31 and 2^31

def randomrange3():  
    ''' 
    generate a random integer in {0, 1, 2} 
    by utilizing a 32-bit random number generator : random32()
    '''
    return ((random32() & 0x3fffffff) * 3) >> 30


# we generate ternary polynomials of degre at most with d1 one and d2 minus one
def T(d1,d2,N):
    import random
    Zx.<x> = ZZ[]    
    a = d1*[1]
    b = d2*[-1]
    c = (N+2-d1-d2)*[0] #the length must be N+1 to get a polynomial of degree at most N
    L = flatten([a,b,c])
    random.shuffle(L) 
    return L,Zx(L)
    
def CenterLift(f,q,N):    
    f_balanced = list(   ((f[i]+q//2)%q) -q//2  for i in range(N)   )
    return Zx(f_balanced)

def reduce_mod_PhiN_and_modq(f):
    return Zx(S(f).lift())%q

def reduce_mod_DN_and_modq(f): # if f=G*F then we get the multiplication of G and F in R/(q,x^N-1) 
    return Zx(R(f).lift())%q

def Convolution_in_S_q(f,g,N,q): 
    PhiN=sum(x^i for i in [0..N-1])
    h = (f*g)%(PhiN)
    return  Zx(h)%q  

def Convolution_in_S(f,g,N):
    PhiN=sum(x^i for i in [0..N-1])
    h = (f*g)%(PhiN)
    #h1 = list( (h[i]%q)   for i in range(N) )
    return  Zx(h) 

def Invertmodprime_and_phiN(f,p,N):        #p must be prime
    phiN = sum(x^i for i in [0..N-1])
    if is_prime(p)==False:
        return "error"
    T = Zx.change_ring(Integers(p)).quotient(PhiN)
    return Zx(lift(1/T(f)))

def invert_mod_2_and_phiN(f):
    return Zx( (1/S2(f)).lift())

def Invertmodpowerofprime_mod_PhiN(f,Q,e,N): # Q prime, e: exponent
    F = invert_mod_2_and_phiN(f)
    if e == 1:      
        return F
    temp_exponent = 2
    while e>0:
        temp = Convolution_in_S(F,f,N)
        F = Convolution_in_S_q(F,2-temp,N,Q^temp_exponent)
        e = floor(e/2)
        temp_exponent = 2*temp_exponent
    return F

def Invert_mod3_and_PhiN(f):
    T = 1/S3(f)
    return T    

def private_keys(N):
    # for L_g we choose d=q/16-1
    # for L_f we choose T_{N-2}
    while True:
        f = Zx([randomrange3()-1 for i in range(N-1)])
        if S3(f).is_unit(): break
    d=q/16-1;g = T(d,d,N-2)
    return f,g[1]

def gen_keys(N,q): 
    f,g=private_keys(N);
    base_of_q = 2
    exponent  = int(log(q,2))
    # N is prime
    # We have set : q = (base_of_q)^(exponent)
    
    try:
        fq=Invertmodpowerofprime_mod_PhiN(f,base_of_q,exponent,N)
    except ZeroDivisionError:
        print("Oops! there is not inverse of f in S_{0}".format(q))
        return _,_,_,_,_,_
    try:
        f3 = Invert_mod3_and_PhiN(f)
    except ZeroDivisionError:
        print("Oops! there is not inverse of f in S_{0}".format(3))
        return _,_,_,_,_,_
    fq = fq%q
    h   = 3*reduce_mod_DN_and_modq(fq * g); # public key, we work mod<q,x^N-1>
    try: 
        hq=Invertmodpowerofprime_mod_PhiN(h,base_of_q,exponent,N)
    except ZeroDivisionError:
        print("Oops! there is not inverse of h in S_{0}".format(q))
        return
    hq = hq%q
    return h,f,Zx(f3.lift()),hq,g,fq

### Encryption
def enc(N,q,msg,h):
    r = Zx([randomrange3()-1 for i in range(N-1)]) #ephemeral key, a ternary of degree <=N-2
    ct1=reduce_mod_DN_and_modq(h*r)  # mod <q,x^N-1>
    ct = reduce_mod_DN_and_modq(ct1 + msg)   # mod <q,x^N-1>
    #print("r={0}\nct1={1}\nmsg={2}\nct={3}".format(r,ct1,msg,ct))
    return ct,r

### decryption
def decryption(ct,f,f3,m):
    a=reduce_mod_DN_and_modq(ct * f)
    a=CenterLift(a,q,N)
    if S3(a*f3)==S3(m):
        print("decryption OK")
    return Zx(S3(a*f3).lift())


In [3]:
# parameters for NTRU-HPS 
# returns N and q
def ntruhps(x):
    if x==1:
        return 509,2048 #ntruhps2048509
    if x==2:
        return 677,2048
    if x==3:
        return 821,4096
N,q=ntruhps(1)

In [4]:
print("N,q=",N,q)
print(q/8 - 1<=2*N/3)
D,PhiN=x^N-1,sum(x^i for i in [0..N-1])

Zx.<x> = ZZ[]
R.<xN> = Zx.quotient(D)
S.<XN> = Zx.quotient(PhiN)

F3 = GF(3); 
F3x.<x3> = F3[]; 
Phi3N = sum(x3^i for i in [0..N-1])
S3.<X3> = F3x.quotient(Phi3N)

F2 = GF(2); F2x.<x2> = F2[]
Phi2N=sum(x2^i for i in [0..N-1])
S2.<X2> = F2x.quotient(Phi2N)


N,q= 509 2048
True


In [5]:
def generate_instance(N,q):
    h,f,f3,hq,g,fq = gen_keys(N=N,q=q)
    
    Zx.<x> = ZZ[]

    D=x^N-1
    PhiN = sum(x^i for i in [0..N-1])\
    
    R.<xN> = Zx.quotient(D)
    S.<XN> = Zx.quotient(PhiN)

    F3 = GF(3); 
    F3x.<x3> = F3[]; 
    Phi3N = sum(x3^i for i in [0..N-1])
    S3.<X3> = F3x.quotient(Phi3N)

    F2 = GF(2); F2x.<x2> = F2[]
    Phi2N=sum(x2^i for i in [0..N-1])
    S2.<X2> = F2x.quotient(Phi2N)

    M = []
    h,f,f3,hq,g,fq = gen_keys(N=N,q=q)
    msg = T(q/16-1,q/16-1,N-2) # Since we want messages of weight q/8-2.
    ct,r=enc(N,q,msg[1],h)     # the ciphertext and the ephemeral key
    m = msg[1]                 # the message as polynomial           
    _=decryption(ct,f,f3,m)    # we check if the decryption is correct
    return f,h,r,ct,msg

#print("N=",N)
#print("k=",kappa)



In [6]:
# construct M_k
# ************* #
def Mk(N,q,k):
    I=identity_matrix(N)
    Zero_Matrix=matrix(N)
    H=-k*I
    B_1=block_matrix([[I,H]])       
    B_2=block_matrix([[Zero_Matrix,q*I]])
    M_NTRU=block_matrix([[B_1],[B_2]])
    return M_NTRU

def u_vector(h,r,kappa):
    # u : the unknown vector such that u(x)=-kappa(h(x)*r(x)) mod q
    u_list = [ x%q for x in R(-kappa*r*h).list()]
    u_vector = vector(u_list)     # write u as sage vector
    return u_vector

def betta(kappa,ct,N):
    b      = kappa*ct % q
    Blist  = b.coefficients(sparse=False)
    return Blist

def generate_u(kappa,h,r,ct,N):
    '''
    In first and second step we construc the vector E, we shall use to build out target vector.
    In this step we shall set E_i = u_i for all i in I_1. In this step we also 
    compute I1, i.e. the indices i, where u_i is exactly computed. 
    '''
    I1=[]
    M_k = Mk(N,q,kappa)
    u = u_vector(h,r,kappa)    
    Blist = betta(kappa,ct,N)
    u_bin = [bin(x)[2:] for x in list(u)]
    u_padded  = [pad_binary(x) for x in u_bin]
    return u_padded

def count_those_with_11(u_padded):
        # List to hold strings that start with '11'
    starts_with_11 = []

    # Check each string in the original list
    for binary in u_padded:
        if binary.startswith('11'):
            starts_with_11.append(binary)

    return len(starts_with_11)

In [19]:
def find_first_one_position(binary_string):
    return binary_string.find('1')

for i in range(25):
    import numpy as np
    if q==2048:kappa,ell=550,bits(q)-1
    entropy = []
    # List to store the positions of the first '1' in each string
    f,h,r,ct,msg=generate_instance(N,q)
    u_padded=generate_u(kappa,h,r,ct,N)
    first_one_positions = [find_first_one_position(bs)+1 for bs in u_padded]
    a=[(ell-i) for i in first_one_positions if i>1]     # we know the length a.k.a we know enough bits
    b= [(ell-i-1) for i in first_one_positions if i==1] # we know two bits
    s = sum(a+b)
    entropy.append(s)
print(np.mean(entropy))
entropy_of_oracle=np.mean(entropy)
print(ell*N*0.77)

decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
decryption OK
4348.0
4311.23000000000


In [9]:
entropy_of_the_msg= N*log(3,2).n()
entropy_of_oracle,entropy_of_the_msg

(4340.0, 806.745912867069)

In [10]:
entropy_of_oracle/entropy_of_the_msg

5.379636798625